In [1]:
import psycopg2
import pandas as pd
from tqdm import tqdm
import numpy as np
from pathos.multiprocessing import ProcessingPool as Pool


### Database #################################################################################################################
def get_cursor_raw():
    host = "10.100.100.2"
    database = "stocks"
    user = "mcai_dcb_write"
    port = 5432
    connection = psycopg2.connect(host=host,
                                  database=database,
                                  user=user,
                                  port=port)
    cursor = connection.cursor()
    return connection, cursor

def single_insert(insert_req):
    """ Execute a single INSERT request """
    connection, cursor = get_cursor_raw()
    try:
        cursor.execute(insert_req)
        connection.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        connection.rollback()
        return 1
    connection.close()
##############################################################################################################################


In [2]:
connection, cursor = get_cursor_raw()

In [3]:
start_time = int(1600732800 *1e9)
end_time = int(1600819200 *1e9)

In [4]:
query =  """ SELECT * FROM iex_historical_orderbook WHERE timestamp > {} AND timestamp < {} AND Symbol ='{}'""".format(start_time,end_time,"AAPL")
print(query)

 SELECT * FROM iex_historical_orderbook WHERE timestamp > 1600732800000000000 AND timestamp < 1600819200000000000 AND Symbol ='AAPL'


In [5]:
a=pd.read_sql_query(query,con=connection)


In [11]:
data =a
data.head()

,timestamp,symbol,side,flag,size,price
0,1600782014070451183,AAPL,buy,,100.0,110.93
1,1600782014796098894,AAPL,buy,,0.0,110.93
2,1600782014976343456,AAPL,sell,,100.0,110.99
3,1600782018443879916,AAPL,buy,,100.0,111.03
4,1600782018443949929,AAPL,buy,,0.0,111.03


In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.animation as animation
from IPython.display import HTML
%matplotlib inline

In [ ]:
_tmp_data = data[0:100]

tfig, ax = plt.subplots(figsize=(15, 8))
ax.barh(_tmp_data['price'], _tmp_data['size'])

In [13]:
plt.show()

In [36]:
_tmp_data = data[0:100]
_tmp_data.head()

# _tmp_data['timestamp'] = [int(x) for x in _tmp_data['timestamp'].values/1e9]

,timestamp,symbol,side,flag,size,price
0,1600782014,AAPL,buy,,100.0,110.93
1,1600782014,AAPL,buy,,0.0,110.93
2,1600782014,AAPL,sell,,100.0,110.99
3,1600782018,AAPL,buy,,100.0,111.03
4,1600782018,AAPL,buy,,0.0,111.03


In [39]:
min_price = _tmp_data['price'].values.min()
max_price = _tmp_data['price'].values.max()
print(min_price, max_price)


110.93 111.29


In [33]:
import pandas as pd
import plotly.express as px 

In [41]:
fig = px.bar(_tmp_data, x="price", y="size", color="side",
   animation_frame="timestamp", animation_group="side", 
   range_y=[0,20])
fig.update_layout(
    height=600,
    title_text=''
)
fig.show()